Importy i start klienta

In [125]:
import binance
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
client = binance.Client()

In [126]:
def get_data(asset):
    candles = client.get_klines(symbol=asset, interval='1m')
    df = pd.DataFrame(candles, columns=['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 
									'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 
									'Taker buy quote asset volume', 'Ignore']).astype("float") # bo domyślnie to są stringi xd
    # Wzór na RSI kradziony z https://www.learnpythonwithrune.org/pandas-calculate-the-relative-strength-index-rsi-on-a-stock/
    delta = df['Close'].diff()
    up = delta.clip(lower=0)
    down = -1*delta.clip(upper=0)
    ema_up = up.ewm(com=13, adjust=False).mean()
    ema_down = down.ewm(com=13, adjust=False).mean()
    rs = ema_up/ema_down
    df['RSI'] = 100 - (100/(1 + rs))
    # Wyrzucamy pierwsze 14 dni żeby mieć wiarygodne wartości RSI
    df = df.iloc[14:]
    return df

def plot_data(df):
    # kradzione z https://python.plainenglish.io/a-simple-guide-to-plotly-for-plotting-financial-chart-54986c996682
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Candlestick(x=df['Close time'],
                    open=df['Open'],
                    high=df['High'],
                    low=df['Low'],
                    close=df['Close'], name=symbol))
    fig.add_trace(go.Scatter(x=df['Close time'], 
                            y=df['RSI'], 
                            opacity=0.7, 
                            line=dict(color='blue', width=2), 
                            name='RSI'), secondary_y=True)
    fig.show()

In [127]:
plot_data(get_data("ATOMUSDT"))